# Spatial normalization of fluorescence to account for differences in illumination brightness

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import pandas as pd

import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
from scipy import stats
from scipy.spatial import distance
import numpy as np
from scipy import interpolate
from pathlib import Path
from sklearn.linear_model import LinearRegression
import matplotlib as mpl
from bsccm import BSCCM

bsccm = BSCCM(str(Path.home()) + '/BSCCM_local/BSCCM/')


In [170]:
def lowess_2d(y_pos, x_pos, z_val, yx_query_points,
              alpha=0.1, weight_fn='tricubic', window_sigma=100):
   
    distances = np.linalg.norm(yx_query_points[:, None] - 
                       np.stack([y_pos, x_pos], axis=1)[None], axis=2)

    # find set of closest points to each query point
    closest_indices = np.argsort(distances, axis=1)[..., :int(alpha * y_pos.size)]
    closest_coords = np.stack([y_pos[closest_indices], x_pos[closest_indices]], axis=2)
    closest_targets = z_val[closest_indices]

    # Compute weighting function based on distance from center of window

    weight_distances = np.sqrt(np.sum((closest_coords - yx_query_points[:, None]) ** 2, axis=2))
#     import pdb; pdb.set_trace()
    
    #normalize to 1
    weight_distances /= np.max(weight_distances)
    if weight_fn == 'tricubic':
        weights = (1 - weight_distances ** 3) ** 3
    elif weight_fn == 'gaussian':
        weights = np.exp( -0.5*(weight_distances / window_sigma) ** 2 )
    else:
        raise Exception('unknown weight fn')
    weights = np.sqrt(weights)

    #solve least squares problems and make predicitions
    predictions = []
    for query_point, w, X, y in zip(
        yx_query_points, weights, closest_coords, closest_targets):
        reg = LinearRegression().fit(X, y, w)
        predictions.append(reg.predict(query_point[None]))
    return np.array(predictions)


def plot_gridded_lowess(y_pos, x_pos, fluor, ax, N=25, alpha=None, weight_fn='tricubic',
                       window_sigma=100):
    #do LOWESS on a grid
    yy, xx = np.meshgrid(np.linspace(0, 2056, N), np.linspace(0, 2056, N))
    yx_query_points = np.stack([yy, xx], axis=-1).reshape([-1, 2])

    # compute alpha adaptively so it corresponds to same number cells
    if alpha is None:
        alpha = 2500 / x_pos.shape[0] 
        print('                     {:.2f} \r'.format(alpha), end='')
    predictions = lowess_2d(y_pos, x_pos, fluor, 
              yx_query_points, alpha=alpha, weight_fn=weight_fn, window_sigma=window_sigma)
    predictions = predictions.reshape([N, N])

    #mask out areas with nothing
    count = stats.binned_statistic_2d(y_pos, x_pos, fluor, bins=N, statistic='count').statistic
    predictions[count == 0] = np.nan
    
    im = ax.imshow(predictions, cmap='inferno')
    plt.colorbar(im, ax=ax)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])
    
def show_spatial_histograms(channel_names, figsize=(9,7), N=25, batch=0):
    replicate = 0
    antibodies_list = bsccm.index_dataframe['antibodies'].unique()

    fig, ax = plt.subplots(len(antibodies_list), len(channel_names), figsize=figsize)
    for i, antibodies in enumerate(antibodies_list):
        mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
        mask_indices = np.flatnonzero(mask)

        for j, channel in enumerate(channel_names):


            if j == 0:
                ax[i, j].set_ylabel(antibodies)
            y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
            x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
            fluor = bsccm.surface_marker_dataframe.loc[mask_indices, channel].to_numpy()
            stat = stats.binned_statistic_2d(y_pos, x_pos, fluor, bins=N, 
                                             statistic=
                                            'mean'
#                                                  lambda x: np.percentile(x, 50)
                                            ).statistic

            contrast_max = np.nanpercentile(stat, 95)
            im = ax[i, j].imshow(stat, cmap='inferno', vmax=contrast_max)

            plt.colorbar(im, ax=ax[i, j])
            ax[i, j].set_yticklabels([])
            ax[i, j].set_xticklabels([])
            ax[i, j].set_yticks([])
            ax[i, j].set_xticks([])
    fig.suptitle('Batch {}'.format(batch))

In [3]:
bsccm.surface_marker_dataframe

,global_index,Fluor_690-_total_raw,Fluor_690-_background,Fluor_627-673_total_raw,Fluor_627-673_background,Fluor_585-625_total_raw,Fluor_585-625_background,Fluor_550-570_total_raw,Fluor_550-570_background,Fluor_500-550_total_raw,Fluor_500-550_background,Fluor_426-446_total_raw,Fluor_426-446_background
0,0,90.205154,74.643562,111.173241,92.166618,91.910187,77.581612,60.964123,58.214993,141.014313,119.545967,69.545181,44.074398
1,1,82.237915,75.487129,104.451164,94.646675,88.273483,79.380478,62.375282,60.246677,137.098679,121.153320,57.115574,43.622066
2,2,80.798347,74.069023,103.424004,92.087128,87.543541,77.186424,59.857498,57.525318,136.572571,118.782181,56.753601,43.377934
3,3,79.028641,74.433380,92.806908,83.702690,78.677719,69.564919,46.312710,44.212730,120.823174,105.807922,51.623005,41.784725
4,4,82.644333,77.617538,98.803795,88.936920,83.008171,73.497032,52.991127,50.805092,129.006073,112.741722,51.924274,42.486279
...,...,...,...,...,...,...,...,...,...,...,...,...,...
412936,412936,74.466263,73.023483,75.277222,69.873268,63.614056,57.465630,51.774380,50.261669,98.392014,87.642433,37.836796,35.951908
412937,412937,75.483070,73.832535,76.499336,71.274117,64.562302,58.331261,56.651958,55.158417,99.938904,90.301842,36.906765,35.732674
412938,412938,72.927307,71.721359,72.462761,68.135788,60.908398,56.115700,51.587204,50.231682,93.220558,85.541161,37.014164,35.643566
412939,412939,70.676392,69.221497,78.399567,72.564072,61.523308,55.135784,56.591564,54.959549,100.411942,89.743141,37.896725,35.698162


## Plan
 - Visualize current state
 - Compute median of background measurements to get slide specific, channel_specific background
 - Compute median of cells to get slide specific, channel_specific, foreground
 - Compute Loess of each one to get a continuous measurement
 - Subtract background from foregound and normalize to get shading correction
 - final result: foreground minus background, divide by normalized shading, visualize result


## Visualize spatial histograms of raw brightness

In [182]:
%matplotlib widget

import matplotlib.pyplot as plt

channel_names = [
    'Fluor_426-446_total_raw',
    'Fluor_500-550_total_raw',
    'Fluor_550-570_total_raw',
    'Fluor_585-625_total_raw',
    'Fluor_627-673_total_raw',
    'Fluor_690-_total_raw',
    ]

show_spatial_histograms(channel_names, figsize=(36, 28), batch=0)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_histograms0.pdf', 
            transparent=True)

show_spatial_histograms(channel_names, figsize=(36, 28), batch=1)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_histograms1.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize spatial histograms of backgrounds

In [183]:
background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']

show_spatial_histograms(background_names, figsize=(36, 28), batch=0)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_backgrounds0.pdf', 
            transparent=True)

show_spatial_histograms(background_names, figsize=(36, 28), batch=1)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_backgrounds1.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize computed smooth function of background intensity pattern

In [176]:
replicate = 0
antibodies_list = bsccm.index_dataframe['antibodies'].unique()

background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']

for batch in range(2):
    fig, ax = plt.subplots(len(antibodies_list), len(background_names), figsize=(32,40))
    
    for i, antibodies in enumerate(antibodies_list):
        print('{} {}\r'.format(batch, antibodies))
        mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
        mask_indices = np.flatnonzero(mask)

        for j, channel in enumerate(background_names):
            if j == 0:
                ax[i, j].set_ylabel(antibodies)
            y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
            x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
            fluor = bsccm.surface_marker_dataframe.loc[mask_indices, background_names[j]].to_numpy()
            
            plot_gridded_lowess(y_pos, x_pos, fluor, ax[i, j], alpha=2500 / x_pos.shape[0] )
    fig.suptitle('Batch {}'.format(batch))
    plt.savefig('/home/henry/leukosight_data/figures/demixing/brightness_backgrounds_lowess{}.pdf'.format(batch), 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 CD45
0 CD123
0 unstained
0 CD19
0 CD56
0 all
0 CD14
0 CD16
0 HLA-DR
0 CD3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1 CD45
1 CD123
1 unstained
1 CD19
1 CD56
1 all
1 CD14
1 CD16
1 HLA-DR
1 CD3


### 1) Go through all slides, compute the background for each, and store lowess background subtracted values
This one takes a long time to run

In [11]:
background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']


antibodies_list = bsccm.index_dataframe['antibodies'].unique()

for batch in range(2):
    for i, antibodies in enumerate(antibodies_list):
        print(antibodies)
        for replicate in range(2):
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)
            if mask_indices.size == 0:
                continue

            for j, channel in enumerate(background_names):            
                y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor = bsccm.surface_marker_dataframe.loc[mask_indices, channel].to_numpy()
                
                yx_query_points = np.stack([y_pos, x_pos], axis=-1)

                # compute alpha adaptively so it corresponds to same number cells
                alpha = 2500 / x_pos.shape[0] 
                predictions = lowess_2d(y_pos, x_pos, fluor, 
                          yx_query_points, alpha=alpha, weight_fn='tricubic')
                bsccm.surface_marker_dataframe.loc[mask_indices, channel + '_lowess'] = predictions            
                

home = str(Path.home())
data_root = home + '/BSCCM_local/BSCCM/'
bsccm.surface_marker_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)

CD45
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD123
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
unstained
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD19
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD56
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
all
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD14
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Flu

### 2) Compute final normalized fluorescence

Pool over markers (subset of 1000 from each slide?), and compute LOWESS over background-subtracted cells divided by mean intensity

    a) Visualize the computed LOWESS
    b) Use it to compute final normalized fluorescence
    
Output: normalized fluorescence


Compile a bunch of measurements of background subtracted brightness for the purposes of creating a shading correction
Since the brightest cells will give the most accurate signal for this, do a spatially local thresholding to only include 
the top 20% brightness cells for each location in the slide

In [178]:


background_names_lowess = ['Fluor_426-446_background_lowess',
 'Fluor_500-550_background_lowess',
 'Fluor_550-570_background_lowess',
 'Fluor_585-625_background_lowess',
 'Fluor_627-673_background_lowess',
 'Fluor_690-_background_lowess']

channel_names = [
    'Fluor_426-446',
    'Fluor_500-550',
    'Fluor_550-570',
    'Fluor_585-625',
    'Fluor_627-673',
    'Fluor_690-',
    ]

antibodies_list = bsccm.index_dataframe['antibodies'].unique()

data_by_channel = {c: {'x_pos': [],
                     'y_pos': [],
                     'background_sub_fluor': [],

                      } for c in channel_names}

for batch in [0, 1]:
    for i, antibodies in enumerate(antibodies_list):

        print(antibodies)
        for replicate in range(2):
            if antibodies == 'unstained' and batch == 1 and replicate == 1:
                #This one has the fieldstop positioned slightly differently and so is not to be trusted
                continue
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)
            if mask_indices.size == 0:
                continue
                
            

            for ch_index in range(len(background_names_lowess)):
                y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor_foreground = bsccm.surface_marker_dataframe.loc[mask_indices, 
                                                           channel_names[ch_index] + '_total_raw'].to_numpy()
                fluor_background_lowess = bsccm.surface_marker_dataframe.loc[mask_indices, 
                                                           background_names_lowess[ch_index]].to_numpy()
                fluor_background_subtracted = fluor_foreground - fluor_background_lowess
                
                # apply spatial threshold: only look at top 20% of brightest cells at each spatial position
                computed = stats.binned_statistic_2d(y_pos, x_pos, fluor_background_subtracted, bins=20, 
                                                             statistic=    lambda x: np.percentile(x, 80),
                                                     expand_binnumbers=True
                                                    )
                spatial_threshold_mask = [f > computed.statistic[computed.binnumber[0, i] - 1,
                                                                 computed.binnumber[1, i] - 1] for i, f in enumerate(fluor_background_subtracted)]
                fluor_background_subtracted = fluor_background_subtracted[spatial_threshold_mask]
                x_pos = x_pos[spatial_threshold_mask]
                y_pos = y_pos[spatial_threshold_mask]
                
                # limit to 2000 per slide to make comptuation easier
                if x_pos.size > 2000:
                    selected_indices = np.random.choice(np.arange(x_pos.size), 2000, replace=False)
                    fluor_background_subtracted = fluor_background_subtracted[selected_indices]
                    x_pos = x_pos[selected_indices]
                    y_pos = y_pos[selected_indices]
                
                
                data_by_channel[channel_names[ch_index]]['x_pos'].append(x_pos)
                data_by_channel[channel_names[ch_index]]['y_pos'].append(y_pos)
                data_by_channel[channel_names[ch_index]]['background_sub_fluor'].append(fluor_background_subtracted) 


for channel in channel_names:
    data_by_channel[channel]['x_pos'] = np.concatenate(data_by_channel[channel]['x_pos'])
    data_by_channel[channel]['y_pos'] = np.concatenate(data_by_channel[channel]['y_pos'])
    data_by_channel[channel]['background_sub_fluor'] = np.concatenate(data_by_channel[channel]['background_sub_fluor'])


CD45
CD123
unstained
CD19
CD56
all
CD14
CD16
HLA-DR
CD3
CD45
CD123
unstained
CD19
CD56
all
CD14
CD16
HLA-DR
CD3


### Visualize spatial histograms of background subtracted

In [179]:
fig, ax = plt.subplots(1, 6, figsize=(36,8))

N = 25

for c_index, channel in enumerate(channel_names):
    print(channel)
    y_pos = data_by_channel[channel]['y_pos']
    x_pos = data_by_channel[channel]['x_pos']
    
    fluor_foreground = bsccm.surface_marker_dataframe.loc[selected_indices, 
                                                           channel_names[ch_index] + '_total_raw'].to_numpy()
    fluor_background_lowess = bsccm.surface_marker_dataframe.loc[selected_indices, 
                                                           background_names_lowess[ch_index]].to_numpy()
    
    fluor = data_by_channel[channel]['background_sub_fluor']
    
    stat = stats.binned_statistic_2d(y_pos, x_pos, fluor, bins=N, 
                                         statistic= 'mean',
#                                                  statistic=    lambda x: np.percentile(x, 90),
                                         expand_binnumbers=True
                                        ).statistic
    
    contrast_max = np.nanpercentile(stat, 95)
    contrast_min = np.nanpercentile(stat, 5)
    im = ax[c_index].imshow(stat, cmap='inferno', vmin= contrast_min, vmax=contrast_max)
    ax[c_index].set_axis_off()
    plt.colorbar(im, ax=ax[c_index])
    ax[c_index].set_title(channel)

plt.savefig('/home/henry/leukosight_data/figures/demixing/brightness_background_subtracted.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fluor_426-446
Fluor_500-550
Fluor_550-570
Fluor_585-625
Fluor_627-673
Fluor_690-


### Visualize LOWESS
This is the factor you'll divide by to do shadign correction

In [180]:
#Normalize data before computing LOWESS
fig, ax = plt.subplots(1, 6, figsize=(32,8))

N = 25

for c_index, channel in enumerate(channel_names):
    print(channel)
    y_pos = data_by_channel[channel]['y_pos']
    x_pos = data_by_channel[channel]['x_pos']
    
#     fluor_foreground = bsccm.surface_marker_dataframe.loc[selected_indices, 
#                                                            channel_names[ch_index] + '_total_raw'].to_numpy()
#     fluor_background_lowess = bsccm.surface_marker_dataframe.loc[selected_indices, 
#                                                            background_names_lowess[ch_index]].to_numpy()
    
    fluor = data_by_channel[channel]['background_sub_fluor']
    
    # normalize 
    fluor = fluor / np.mean(fluor)
    
    alpha = 2500 / x_pos.shape[0] 
    plot_gridded_lowess(y_pos, x_pos, fluor, ax[c_index], N=N, weight_fn='tricubic', alpha=alpha)
    ax[c_index].set_title(channel)
plt.savefig('/home/henry/leukosight_data/figures/demixing/shading.pdf', 
            transparent=True)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fluor_426-446
Fluor_500-550
Fluor_550-570
Fluor_585-625
Fluor_627-673
Fluor_690-


### Use normalized LOWESS to do shading correction

In [157]:
channel_names = [
    'Fluor_426-446',
    'Fluor_500-550',
    'Fluor_550-570',
    'Fluor_585-625',
    'Fluor_627-673',
    'Fluor_690-',
    ]

background_names_lowess = ['Fluor_426-446_background_lowess',
 'Fluor_500-550_background_lowess',
 'Fluor_550-570_background_lowess',
 'Fluor_585-625_background_lowess',
 'Fluor_627-673_background_lowess',
 'Fluor_690-_background_lowess']

for i in range(6):            
    print(channel_names[i])
    
    #Do one marker at a time to save on memory
    batch_size = 1000
    for chunk_index in range(bsccm.index_dataframe.global_index.size // batch_size + 1):
        print('{} of {}    \r'.format(chunk_index, bsccm.index_dataframe.global_index.size // batch_size + 1), end='')
        mask = np.logical_and(bsccm.index_dataframe.global_index >= chunk_index * batch_size,
                              bsccm.index_dataframe.global_index < (chunk_index + 1) * batch_size)
        
        indices = np.flatnonzero(mask)
        
        
        # 1) Load background subtracted
        y_pos = bsccm.index_dataframe.loc[indices, 'position_in_fov_y_pix'].to_numpy()
        x_pos = bsccm.index_dataframe.loc[indices, 'position_in_fov_x_pix'].to_numpy()
        foreground = bsccm.surface_marker_dataframe.loc[indices, channel_names[i] + '_total_raw'].to_numpy()
        background = bsccm.surface_marker_dataframe.loc[indices, background_names_lowess[i]].to_numpy()

        background_corrected = foreground - background

        #a subset of the data in that channel, so whole thing doesnt take forever
        channel_ref_examples = data_by_channel[channel_names[i]]['background_sub_fluor']
        channel_ref_y_pos = data_by_channel[channel]['y_pos']
        channel_ref_x_pos = data_by_channel[channel]['x_pos']

        #TODO:
        # 2a) Compute LOWESS on normalized data

        yx_query_points = np.stack([y_pos, x_pos], axis=-1)


        #compute the shading -- lowess over a representative set of background examples
        2500 / channel_ref_x_pos.shape[0] 
        shading = lowess_2d(channel_ref_y_pos, channel_ref_x_pos, channel_ref_examples, 
                  yx_query_points, alpha=alpha, weight_fn='tricubic')
        bsccm.surface_marker_dataframe.loc[indices, channel_names[i] + '_shading'] = shading
        
    #compute normalized version for all at once
    shading = bsccm.surface_marker_dataframe[channel_names[i] + '_shading'].to_numpy()
    #normalize to avoid numerical errors
    shading = shading / np.mean(shading)

    # 2b) Divide by shading correction factor
    foreground = bsccm.surface_marker_dataframe[channel_names[i] + '_total_raw']
    background = bsccm.surface_marker_dataframe[background_names_lowess[i]]

    corrected = (foreground - background) / shading

    # 3) Store result
    bsccm.surface_marker_dataframe[channel_names[i] + '_shading_corrected'] = corrected

#Resave
home = str(Path.home())
data_root = home + '/BSCCM_local/BSCCM/'
bsccm.surface_marker_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)

Fluor_690-
Fluor_690-    
Fluor_690-    
Fluor_690-    
Fluor_690-    
Fluor_690-    


In [159]:
bsccm.surface_marker_dataframe

,global_index,Fluor_690-_total_raw,Fluor_690-_background,Fluor_627-673_total_raw,Fluor_627-673_background,Fluor_585-625_total_raw,Fluor_585-625_background,Fluor_550-570_total_raw,Fluor_550-570_background,Fluor_500-550_total_raw,...,Fluor_500-550_shading,Fluor_500-550_shading_corrected,Fluor_550-570_shading,Fluor_550-570_shading_corrected,Fluor_585-625_shading,Fluor_585-625_shading_corrected,Fluor_627-673_shading,Fluor_627-673_shading_corrected,Fluor_690-_shading,Fluor_690-_shading_corrected
0,0,90.205154,74.643562,111.173241,92.166618,91.910187,77.581612,60.964123,58.214993,141.014313,...,80.666408,-9.546087,10.115647,0.244206,34.954990,4.599937,33.349353,17.730120,21.470533,14.561309
1,1,82.237915,75.487129,104.451164,94.646675,88.273483,79.380478,62.375282,60.246677,137.098679,...,78.718180,-17.419873,9.876530,-0.541537,33.993266,-1.870488,32.534823,7.757642,20.129025,5.836843
2,2,80.798347,74.069023,103.424004,92.087128,87.543541,77.186424,59.857498,57.525318,136.572571,...,78.713359,-12.872697,9.765018,-0.042141,34.339840,0.468060,32.555925,9.668359,20.685243,5.601074
3,3,79.028641,74.433380,92.806908,83.702690,78.677719,69.564919,46.312710,44.212730,120.823174,...,86.203365,-7.148947,10.419661,-3.929642,36.784052,-1.453486,35.674436,5.704854,21.041925,6.570466
4,4,82.644333,77.617538,98.803795,88.936920,83.008171,73.497032,52.991127,50.805092,129.006073,...,82.367954,-12.806695,10.256053,-4.428715,36.088059,-1.035731,35.341019,8.155553,21.453345,8.231029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412936,412936,74.466263,73.023483,75.277222,69.873268,63.614056,57.465630,51.774380,50.261669,98.392014,...,80.195842,3.854952,9.851740,0.801560,35.055580,5.014530,33.796833,2.777335,21.767867,3.376433
412937,412937,75.483070,73.832535,76.499336,71.274117,64.562302,58.331261,56.651958,55.158417,99.938904,...,76.685799,1.525111,9.814024,0.693378,33.739942,4.546722,32.451605,2.060166,20.287460,3.327853
412938,412938,72.927307,71.721359,72.462761,68.135788,60.908398,56.115700,51.587204,50.231682,93.220558,...,79.952670,0.479350,9.736240,0.633300,34.247302,3.530663,32.423705,1.562294,19.844133,3.586604
412939,412939,70.676392,69.221497,78.399567,72.564072,61.523308,55.135784,56.591564,54.959549,100.411942,...,82.330931,2.160017,10.218266,1.706385,35.313222,0.771515,34.760057,2.876208,22.267203,-2.612637


### Validate the final result by looking at spatial histograms of normalized fluorescence
Which should no longer have an obvious spatial distribution

In [181]:
corrected_channel_names = [
    'Fluor_426-446_shading_corrected',
    'Fluor_500-550_shading_corrected',
    'Fluor_550-570_shading_corrected',
    'Fluor_585-625_shading_corrected',
    'Fluor_627-673_shading_corrected',
    'Fluor_690-_shading_corrected',
    ]

show_spatial_histograms(corrected_channel_names, N=25, figsize=(36, 28), batch=0)

plt.savefig('/home/henry/leukosight_data/figures/demixing/shading_corrected_brightness0.pdf', 
            transparent=True)

show_spatial_histograms(corrected_channel_names, N=25, figsize=(36, 28), batch=1)

plt.savefig('/home/henry/leukosight_data/figures/demixing/shading_corrected_brightness1.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …